In [1]:
# Data Preprocessing - version 1

In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, ensemble
from sklearn import model_selection
from xgboost import XGBClassifier
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def read_file(filename):
    raw_data = pd.read_csv(filename)
    data = raw_data.copy()
    return data

In [3]:
# drop features
def select_same_resp_feature(X_train):
    same_resp_feature = []
    for feature in X_train.columns:
        if len(np.unique(X_train[feature])) == 1:
            same_resp_feature.append(feature)
    print(len(same_resp_feature))
    return same_resp_feature

def select_allocation_flag(X_train):
    allocation_flag_feature = []
    for feature in X_train.columns:
        if feature[:2] == 'PX' or feature[:2] == 'HX':
            #print(feature)
            allocation_flag_feature.append(feature)
    other_alloc = ['PRCITFLG', 'PRWERNAL', 'PRHERNAL']
    allocation_flag_feature = allocation_flag_feature + other_alloc
    print(len(allocation_flag_feature))
    return allocation_flag_feature

# drop useless features together
def drop_features(X_train):
    same_resp_feature = select_same_resp_feature(X_train)
    allocation_flag_feature = select_allocation_flag(X_train)
    drop_features = list(set(same_resp_feature + allocation_flag_feature))
    X_train.drop(drop_features, axis=1, inplace=True)
    print(X_train.shape)
    

In [4]:
def train_data_preprocessing(filename):
    # read in the train file
    train_data = read_file(filename)
    y_train = train_data['target']
    X_train_raw = train_data.drop('target', axis=1)

    # no dropping raw data -> np array
    X_train_raw_arr = X_train_raw.values
    y_train_arr = y_train.values
    
    # drop features -> np array
    to_drop_features = select_same_resp_feature(X_train_raw)
    X_train_drop = X_train_raw.drop(to_drop_features, axis=1)
    X_train_drop_arr = X_train_drop.values
    
    return X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features

def test_data_preprocessing(filename, to_drop_features):
    test_data = read_file(filename)
    X_test_raw = test_data
    X_test_raw_arr = X_test_raw.values

    X_test_drop = X_test_raw.drop(to_drop_features, axis=1)
    X_test_drop_arr = X_test_drop.values
    
    return X_test_raw_arr, X_test_drop_arr

In [ ]:
# categorical features
train_data = read_file("train_2008.csv")
test_data = read_file("test_2008.csv")

same_resp_feature = select_same_resp_feature(train_data)
train_data.drop(same_resp_feature, axis=1, inplace=True)
test_data.drop(same_resp_feature, axis=1, inplace=True)

categorical_feats = ['GEREG', 'HUBUS', 'PTDTRACE', 'PENATVTY', 'PUABSOT', 'PEIO1COW', 
                     'HUFINAL', 'GESTCEN', 'GESTFIPS', #'PEIO1ICD', 'PEIO2ICD', 
                     'PRCITSHP', 'PUDIS', 'PRABSREA', 'PRWKSTAT', 'HUPRSCNT', 
                     'PERRP', 'GTCBSAST', 'PRMJOCGR', 'HRHTYPE', ]

for feat in categorical_feats:
    train_data[feat] = train_data[feat].apply(lambda x: -1 if x < 0 else x)
    test_data[feat] = test_data[feat].apply(lambda x: -1 if x < 0 else x)

# Now dummy these features
train_dummy_df = pd.DataFrame()
test_dummy_df = pd.DataFrame()

for var in categorical_feats:
    train_dummy_vars = pd.get_dummies(train_data[var], prefix=var)
    train_dummy_df = pd.concat([train_dummy_df, train_dummy_vars], axis=1)
    
    test_dummy_vars = pd.get_dummies(test_data[var], prefix=var)
    test_dummy_df = pd.concat([test_dummy_df, test_dummy_vars], axis=1)
    
# Drop the original categorical variables
train_data.drop(categorical_feats, axis=1, inplace=True)
test_data.drop(categorical_feats, axis=1, inplace=True)
# Add dummy vars to the data
train_data = pd.concat([train_data, train_dummy_df], axis=1)
test_data = pd.concat([test_data, test_dummy_df], axis=1)
# Now the train and test data have different numbers of features -> fix it!
feats_to_add_to_train = [f for f in test_data.columns if f not in train_data.columns]
feats_to_add_to_test = [f for f in train_data.columns if f not in test_data.columns 
                        and f != 'target']

for feat in feats_to_add_to_train:
    train_data[feat] = 0
for feat in feats_to_add_to_test:
    test_data[feat] = 0

print(train_data.values.shape)
print(test_data.values.shape)

y_train = train_data['target']
X_train = train_data.drop('target', axis=1)
X_test = test_data

#seed=7
X = X_train.values
Y = y_train.values
X_t = X_test.values
#test_size = 0.33
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
clf = XGBClassifier(max_depth=6) 
clf.fit(X, Y)
y_train_pred = clf.predict_proba(X)[:, 1]
y_test_pred = clf.predict_proba(X_t)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
pd.DataFrame(y_test_pred_prob).to_csv("gbdt_test2008.csv")

In [18]:
pd.DataFrame(y_tesdt_pred).to_csv("gbdt_test2008.csv")

In [15]:
X_test = read_file("train_2008.csv")
X_test.drop(same_resp_feature, axis=1, inplace=True)
for feat in X_test.columns:
    X_test[feat] = X_test[feat].apply(lambda x: -1 if x < 0 else x)
for feat in categorical_feats:
    X_test = pd.get_dummies(X_test, columns=[feat])
assert (X_train.columns == test_data.columns).all()
print(X.shape)
print(X_test.values.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [6]:
# numerical features

In [7]:
# one-hot encoding

In [8]:
# raw data for random forest

In [9]:
train_path = "train_2008.csv"
test_path = "test_2008.csv"
X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features = train_data_preprocessing(train_path)
X_test_raw_arr, X_test_drop_arr = test_data_preprocessing(test_path, to_drop_features)

print(X_train_raw_arr.shape)
print(y_train_arr.shape)
print(X_train_drop_arr.shape)
print(X_test_raw_arr.shape)
print(X_test_drop_arr.shape)

15
(64667, 382)
(64667,)
(64667, 367)
(16000, 382)
(16000, 367)


In [10]:
def run_test(X_train_arr, y_train_arr, num_folds, C, X_test_arr, clf):
    clf.fit(X_train_arr, y_train_arr)
    y_train_pred_prob = clf.predict_proba(X_train_arr)[:, 1]
    train_auc = metrics.roc_auc_score(y_train_arr, y_train_pred_prob)
    print("train auc: ", train_auc)
    y_test_pred_prob = clf.predict_proba(X_test_arr)[:, 1]
    return train_auc, y_test_pred_prob

In [11]:
# calculate auc score for cross_validation
def cross_validation(num_folds, X_train_arr, clf):
    kf = model_selection.KFold(n_splits=num_folds)
    auc = []
    for train_index, test_index in kf.split(X_train_arr):
        print("1---")
        X_train_cv, X_test_cv = X_train_arr[train_index], X_train_arr[test_index]
        y_train_cv, y_test_cv = y_train_arr[train_index], y_train_arr[test_index]
        clf.fit(X_train_cv,y_train_cv)
        y_pred_prob = clf.predict_proba(X_test_cv)[:, 1]
        auc.append(metrics.roc_auc_score(y_test_cv, y_pred_prob))
        print("2---")
    return auc

In [12]:
# logistic regression
def logistic_reg(X_train_arr, num_folds, C):
    clf = linear_model.LogisticRegression(penalty='l1', C=C, solver='liblinear')
    #clf = linear_model.LogisticRegression()
    #auc = cross_validation(num_folds, X_train_arr, clf)
    #return auc, clf
    return clf

In [11]:
# raw data for logistic regression

# tune the hyperparameter C has little influence
# current C =  0.3 , auc =  [0.7663607035376859, 0.7721732996821118, 0.7687268216387099]
# current C =  0.475 , auc =  [0.7662418721269123, 0.7712735791351473, 0.768680422577126]
# current C =  0.6499999999999999 , auc =  [0.7661768150447318, 0.7713118201969408, 0.7683741910137993]
# current C =  0.825 , auc =  [0.7661509304142629, 0.7711936392253098, 0.7682658368117301]
# current C =  1.0 , auc =  [0.766072324965325, 0.7707366180248577, 0.7682738111247072]

# X_train_raw_arr, y_train_raw_arr, X_train_drop_arr = data_preprocessing(train_path, to_drop_features)
# X_test_raw_arr, y_test_raw_arr, X_test_drop_arr = data_preprocessing(test_path, to_drop_features)
# num_folds = 3
# C_para = np.linspace(0.3, 1, 5)
# for C in C_para:
#     auc = logistic_reg(X_train_raw_arr, num_folds, C)
#     print("current C = ", C, ", auc = ", auc)

# ??? how can I ensemble LR models? will ensemble here improves the performance?

In [14]:
# run logistic regression
X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features = train_data_preprocessing(train_path)
X_test_raw_arr, X_test_drop_arr = test_data_preprocessing(test_path, to_drop_features)
num_folds = 3
C = 0.3
clf_lr = logistic_reg(X_train_raw_arr, num_folds, C)
train_auc, y_test_pred_prob = run_test(X_train_raw_arr, y_train_arr, num_folds, C, X_test_raw_arr, clf_lr)
pd.DataFrame(y_test_pred_prob).to_csv("lr_test2008.csv")
print("done!")

15
train auc:  0.7764389064762474
done!


In [32]:
# gbdt
def gbdt(X_train_arr, num_folds):
    #clf_gbdt = ensemble.GradientBoostingClassifier(random_state=10)
    clf_gbdt = ensemble.GradientBoostingClassifier(n_estimators=50, random_state=10, min_samples_leaf=20) 
    #clf_gbdt = ensemble.GradientBoostingClassifier(learning_rate=0.5, min_samples_split=300, min_samples_leaf=20,max_depth=5,max_features='sqrt', subsample=0.8,random_state=10)
    #auc = cross_validation(num_folds, X_train_arr, clf_gbdt)
    #print("auc = ", auc)
    return clf_gbdt

In [21]:
# run gbdt
X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features = train_data_preprocessing(train_path)
X_test_raw_arr, X_test_drop_arr = test_data_preprocessing(test_path, to_drop_features)
num_folds = 3
clf_gbdt = gbdt(X_train_raw_arr, num_folds)
train_auc, y_test_pred_prob = run_test(X_train_raw_arr, y_train_arr, num_folds, C, X_test_raw_arr, clf_gbdt)
pd.DataFrame(y_test_pred_prob).to_csv("gbdt_raw_test2008.csv")
print("done!")

15
train auc:  0.8254147729384751
done!


15
train auc:  0.7929522993012791
done!


In [34]:
X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features = train_data_preprocessing(train_path)
X_test_raw_arr, X_test_drop_arr = test_data_preprocessing(test_path, to_drop_features)
param_test = {'min_samples_split':range(200,900,100), 'min_samples_leaf': range(30, 70, 10), 'max_depth': range(5, 10, 1)}
estimator = ensemble.GradientBoostingClassifier(
    learning_rate=0.8,
    #max_features='sqrt',
    random_state=10,
    n_estimators=50
)
gsearch = model_selection.GridSearchCV(estimator, param_grid = param_test, scoring='roc_auc', cv=5)
gsearch.fit(X_train_raw_arr, y_train_arr)
print("best_params: ", gsearch1.best_params_)
print("best_score: ", gsearch1.best_score_)

15


KeyboardInterrupt: 

In [33]:
X_train_raw_arr, y_train_arr, X_train_drop_arr, to_drop_features = train_data_preprocessing(train_path)
X_test_raw_arr, X_test_drop_arr = test_data_preprocessing(test_path, to_drop_features)
num_folds = 3
clf_gbdt = gbdt(X_train_raw_arr, num_folds)
train_auc, y_test_pred_prob = run_test(X_train_raw_arr, y_train_arr, num_folds, C, X_test_raw_arr, clf_gbdt)
pd.DataFrame(y_test_pred_prob).to_csv("gbdt_raw_test2008.csv")
print("done!")

15
train auc:  0.7818926933969366
done!
